<a href="https://colab.research.google.com/github/huyminh1115/Trip-Advisor-Hotel-Project/blob/main/Code/Final_Simple_MLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from wordcloud import WordCloud

# Preprocessing and evaluation
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l1, l2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_data_v2.csv")

In [5]:
data.head()

,Review,Rating,Length,Word_count,cleaned_review,sentiment,tokenized_review,review_vector
0,nice hotel expensive parking got good deal sta...,4,593,87,nice hotel expensive parking got good deal sta...,0.208744,"['nice', 'hotel', 'expensive', 'parking', 'got...",[ 0.26746067 -0.39511007 0.2275819 0.329419...
1,ok nothing special charge diamond member hilto...,2,1689,250,ok nothing special charge diamond member hilto...,0.248633,"['ok', 'nothing', 'special', 'charge', 'diamon...",[-0.13580382 -0.10084884 -0.06840305 0.191020...
2,nice rooms not 4* experience hotel monaco seat...,3,1427,217,nice room experience hotel monaco seattle good...,0.294420,"['nice', 'room', 'experience', 'hotel', 'monac...",[ 0.12907822 -0.4086012 0.00375678 0.077708...
3,"unique, great stay, wonderful time hotel monac...",5,600,89,unique great stay wonderful time hotel monaco ...,0.504825,"['unique', 'great', 'stay', 'wonderful', 'time...",[ 0.14176802 -0.31248108 0.11944489 0.082050...
4,"great stay great stay, went seahawk game aweso...",5,1281,191,great stay great stay went seahawk game awesom...,0.469643,"['great', 'stay', 'great', 'stay', 'went', 'se...",[-0.13615511 -0.08891882 -0.0077215 0.260447...


In [1]:
data['sentiment'] = data['Rating'].apply(lambda x: 1 if x > 3 else 2 if x == 3 else 0)

def convert_to_float_array(s):
    # Bỏ dấu ngoặc rồi tách theo khoảng trắng
    s = s.strip('[]').split()
    # Chuyển từng phần tử trong danh sách thành float
    return [float(x) for x in s]


data['review_vector'] = data['review_vector'].apply(convert_to_float_array)
# Prepare data
X = np.array(data['review_vector'].tolist())
y = data['sentiment']

NameError: name 'data' is not defined

In [7]:
from sklearn.model_selection import KFold

# Initialize 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Save the indices of each fold for reuse
fold_indices = [(train_idx, test_idx) for train_idx, test_idx in kf.split(X)]

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import time

# Hàm tạo mô hình MLP
def create_mlp():
    model = Sequential()
    model.add(Dense(64, input_dim=100, activation='relu'))  # Lớp ẩn đầu tiên với 64 node và hàm kích hoạt ReLU
    model.add(Dense(64, activation='relu'))  # Lớp ẩn thứ hai với 64 node và hàm kích hoạt ReLU
    model.add(Dense(3, activation='softmax'))  # Lớp đầu ra với 3 node (tương ứng 3 lớp), hàm softmax cho phân loại
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Compile mô hình
    return model

In [1]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    gpu_name = tf.config.list_physical_devices('GPU')[0]
    print("GPU in use:", gpu_name)
else:
    print("GPU is not available.")


GPU is available.
GPU in use: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [22]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define EarlyStopping callback to stop training if no improvement after 'patience' epochs
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=30,          # Number of epochs with no improvement to stop training
    restore_best_weights=True  # Restore the best weights at the end of training
)

# Store results for each fold
results = []

for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y), start=1):
    print(f"\nRunning Fold {fold_idx}...")

    # Split data into train and test sets for this fold
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Convert labels to categorical (for multi-class classification)
    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Create a new instance of the MLP model
    model = create_mlp()

    # Train the model
    start_time = time.time()
    # Train the model with EarlyStopping
    history = model.fit(
        X_train, y_train_cat,
        epochs=200,  # Maximum number of epochs
        batch_size=64,
        validation_data=(X_test, y_test_cat),  # Validation data for EarlyStopping monitoring
        callbacks=[early_stopping],  # Include the EarlyStopping callback
        verbose=0  # Display training progress
    )
    elapsed_time = time.time() - start_time

    # Evaluate on the training set
    train_loss, train_accuracy = model.evaluate(X_train, y_train_cat, verbose=0)

    # Evaluate on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

    # Make predictions for metrics calculation
    y_test_pred = np.argmax(model.predict(X_test), axis=1)
    y_train_pred = np.argmax(model.predict(X_train), axis=1)

    # Calculate metrics
    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    train_recall = recall_score(y_train, y_train_pred, average='weighted')

    test_f1 = f1_score(y_test, y_test_pred, average='weighted')
    test_precision = precision_score(y_test, y_test_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')

    # Store results for this fold
    results.append({
        "Fold": fold_idx,
        "Train Accuracy": train_accuracy,
        "Train F1": train_f1,
        "Train Precision": train_precision,
        "Train Recall": train_recall,
        "Test Accuracy": test_accuracy,
        "Test F1": test_f1,
        "Test Precision": test_precision,
        "Test Recall": test_recall,
        "Time": elapsed_time
    })

    # Print fold results
    print(f"Fold {fold_idx} Results:")
    print(f"  Train -> Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}, "
          f"Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"  Test  -> Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}, "
          f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")
    print(f"  Time Taken: {elapsed_time:.2f} seconds\n")

# Calculate and display averaged metrics across all folds
avg_metrics = {
    metric: np.mean([result[metric] for result in results])
    for metric in ["Train Accuracy", "Train F1", "Train Precision", "Train Recall",
                   "Test Accuracy", "Test F1", "Test Precision", "Test Recall", "Time"]
}

print("\nAveraged Results Across All Folds:")
print(f"  Train -> Accuracy: {avg_metrics['Train Accuracy']:.4f}, "
      f"F1: {avg_metrics['Train F1']:.4f}, "
      f"Precision: {avg_metrics['Train Precision']:.4f}, "
      f"Recall: {avg_metrics['Train Recall']:.4f}")
print(f"  Test  -> Accuracy: {avg_metrics['Test Accuracy']:.4f}, "
      f"F1: {avg_metrics['Test F1']:.4f}, "
      f"Precision: {avg_metrics['Test Precision']:.4f}, "
      f"Recall: {avg_metrics['Test Recall']:.4f}")
print(f"  Average Time Taken per Fold: {avg_metrics['Time']:.2f} seconds")


Running Fold 1...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 1 Results:
  Train -> Accuracy: 0.8565, F1: 0.8358, Precision: 0.8357, Recall: 0.8565
  Test  -> Accuracy: 0.8576, F1: 0.8377, Precision: 0.8377, Recall: 0.8576
  Time Taken: 21.92 seconds


Running Fold 2...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 2 Results:
  Train -> Accuracy: 0.8350, F1: 0.7999, Precision: 0.8031, Recall: 0.8350
  Test  -> Accuracy: 0.8336, F1: 0.7983, Precision: 0.7997, Recall: 0.8336
  Time Taken: 15.71 seconds


Running Fold 3...


...

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 8 Results:
  Train -> Accuracy: 0.8361, F1: 0.7947, Precision: 0.7999, Recall: 0.8361
  Test  -> Accuracy: 0.8336, F1: 0.7884, Precision: 0.8140, Recall: 0.8336
  Time Taken: 15.54 seconds


Running Fold 9...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 9 Results:
  Train -> Accuracy: 0.8369, F1: 0.7939, Precision: 0.8199, Recall: 0.8369
  Test  -> Accuracy: 0.8326, F1: 0.7873, Precision: 0.8531, Recall: 0.8326
  Time Taken: 15.51 seconds


Running Fold 10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 10 Results:
  Train -> Accuracy: 0.8523, F1: 0.8257, Precision: 0.8265, Recall: 0.8523
  Test  -> Accuracy: 0.8570, F1: 0.8346, Precision: 0.8298, Recall: 0.8570
  Time Taken: 20.91 seconds


Averaged Results Across All Folds:
  Train -> Accuracy: 0.8395, F1: 0.8051, Precision: 0.8097, Recall: 0.8395
  Test  -> Accuracy: 0.8364, F1: 0.8010, Precision: 0.8083, Recall: 0.8364
  Average Time Taken per Fold: 16.85 seconds
